In [30]:
import tensorflow as tf

In [31]:
from tensorflow.examples.tutorials.mnist import input_data

load MNIST data

In [32]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


create placeholders

In [33]:
X = tf.placeholder(tf.float32, shape=[None, 784])
y_true = tf.placeholder(tf.float32, shape=[None, 10])
hold_prob = tf.placeholder(tf.float32)

create convolutional layers and pooling layers

In [34]:
X_shaped = tf.reshape(X, [-1, 28, 28, 1])
conv1 = tf.layers.conv2d(inputs=X_shaped, filters=32, kernel_size=5, padding="same", activation=tf.nn.relu)

In [35]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=2)

In [36]:
conv2 = tf.layers.conv2d(inputs=pool1, filters=64, kernel_size=5, padding="same", activation=tf.nn.relu)

In [37]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=2)

create dnn part

In [38]:
pool2_shaped = tf.reshape(pool2, [-1, 7*7*64])
dense = tf.layers.dense(inputs=pool2_shaped, units=1024, activation=tf.nn.relu)

In [39]:
dropout = tf.nn.dropout(dense, keep_prob=hold_prob) # use dropout to improve performance

In [40]:
logits = tf.layers.dense(inputs=dropout, units=10)

create loss function using cross entropy

In [41]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits))

create optimizer

In [42]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [43]:
train = optimizer.minimize(cross_entropy)

train and evaluate

In [45]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(101):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train, feed_dict={X:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            y_equal = tf.equal(tf.argmax(y_true, axis=1), tf.argmax(logits, axis=1))
            acc = tf.reduce_mean(tf.cast(y_equal, tf.float32))
            print(sess.run(acc, feed_dict={X:mnist.test.images, y_true: mnist.test.labels, hold_prob:1.0}))
            

0.1221
0.9525
